<a href="https://colab.research.google.com/github/MikelKN/WOAH-2025/blob/main/LLM_prompt_strategies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALL the PROMPTS and STRATEGIES

In [ ]:
# limit to the lenght of token that should be displayed on the daatsets
!pip install openai
from openai import OpenAI

from google.colab import drive
drive.mount('/content/drive')

# Class for Cohere AI
!pip install cohere transformers --quiet
!pip install openai mistral_inference llamaapi>=0.1.36 tqdm fire --quiet


import pandas as pd
import numpy as np
# import kaggle
from tqdm import tqdm
import pandas as pd
import cohere

pd.options.display.max_colwidth = 500

from pathlib import Path
# !pip install openai
from openai import OpenAI

from google.colab import userdata
import os
os.environ["COHERE_PROD"] = userdata.get("COHERE_PROD")
os.environ["NEW_OPEN_AI"] = userdata.get("NEW_OPEN_AI")
# os.environ["OPEN_AI"] = userdata.get("OPEN_AI")


class Cohere_ai:
    def prompt_cohere(self, event, model_name= 'command-r7b-12-2024'):
      prompt_empathy_vanilla = f"Assume the role of an NGO professional specializing in countering online hate speech with empathy and fostering constructive discourse.\
      Your task is to generate a concise, well-reasoned, and compassionate counter-narrative in response to the following comment: {event}. \
      Your responses should closely mirror the knowledge and abilities of an NGO worker. No additional explanations are required. provide the counter narrative only."
      co_api = cohere.Client(COHERE_PROD)
      response = co_api.chat(
          message=prompt_empathy_vanilla,
          model=model_name,
          temperature=0.3
      )
      return response.text

class gpt_4o_model:
    def prompt_gpt4o(self, event, model_name = 'gpt-4o-mini'):
      prompt_empathy_ngo = f"Assume the role of an NGO professional specializing in countering online hate speech with empathy and fostering constructive discourse.\
      Your task is to generate a concise, well-reasoned, and compassionate counter-narrative in response to the following comment: {event}. \
      Your responses should closely mirror the knowledge and abilities of an NGO worker. No additional explanations are required. provide the counter narrative only."
      client = OpenAI(
          api_key = NEW_OPEN_AI
          )
      input = [{"role": "system", "content": "You are a NGO worker and expert in generating compasssionate countrer narrative."}, #vanilla : "You are a helpful assistant." "You are an ngo worker on a mission to mitigate hateful language online."
              {"role": "user","content": prompt_empathy_ngo,}]

      chat_completion = client.chat.completions.create(
          messages=input,
          model=model_name,
          temperature=0.3
      )
      output = chat_completion.choices[0].message.content
      return output

class llama_instruct:
    def prompt_llama(self, event, model_name ="llama3.1-70b"):
      prompt = f"Assume the role of an NGO professional specializing in countering online hate speech with empathy and fostering constructive discourse.\
      Your task is to generate a concise, well-reasoned, and compassionate counter-narrative in response to the following comment: {event}. \
      Your responses should closely mirror the knowledge and abilities of an NGO worker. No additional explanations are required. provide the counter narrative only."

      client = OpenAI(
          api_key = LLama-API-Key,
          base_url = "https://api.llama-api.com")

      input = [{"role": "system", "content": "You are an NGO worker and expert in generating compasssionate countrer narrative." }, #You are a counter narrative expert."You are an ngo worker on a mission to mitigate hateful language online."
              {"role": "user","content": prompt,}]

      chat_completion = client.chat.completions.create(
      model=model_name,
      messages=input,
      temperature = 0.3
      )
      output = chat_completion.choices[0].message.content

      return output

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.3 MB/s eta 0:00:00


# Sentiment and emotion label

In [ ]:
# https://www.kaggle.com/code/ncho11/introduction-to-prompt-engineering-using-mistral

!pip install transformers --quiet
!pip install accelerate --quiet
!pip install bitsandbytes --quiet
!pip install langchain --quiet

import warnings
warnings.filterwarnings('ignore') # avoid warning messages importing packages

os.environ["HF_API_KEY"] = userdata.get("HF_API_KEY")
os.environ["COHERE_API"] = userdata.get("COHERE_API")

HF_TOKEN = userdata.get("HF_API_KEY")
from huggingface_hub import login
login(HF_TOKEN)

import torch
import re
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


mistral_model = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(mistral_model, use_fast= True)
tokenizer.pad_token = tokenizer.eos_token
mistral_model_new = AutoModelForCausalLM.from_pretrained(mistral_model,
                                                         torch_dtype=torch.float16,
                                                         trust_remote_code=True,
                                                         device_map="auto",
                                                         quantization_config=quantization_config)

generation_config = GenerationConfig.from_pretrained(mistral_model)
generation_config.max_new_tokens = 20
generation_config.temperature = 0.1
generation_config.do_sample = True

mistral_pipeline = pipeline(
    "text-generation",
    model=mistral_model_new,
    tokenizer=tokenizer,
    truncation=True,
    return_full_text=False,
    generation_config=generation_config,
)
output_path= 'some path'

class LLM_sentiment:

    def sentiment_mistral_2(self, event):
        prompt = f"""
        As an expert in sentiment analysis, analyze the following '{event}' and determine the dominant sentiment it evokes.
        Reply with strictly one sentiment(lowercase, no quotes, and no punctuations) chosen from: very negative, negative, neutral, positive, or very positive.
        Response should be in the following structured format (strictly one line):
        sentiment: chosen sentiment
        Ensure that the response strictly follows this format, with no additional explanation, commentary, or justification beyond the required output. No other response will be accepted.
        """
        response = mistral_pipeline(prompt)
        output = response[0]['generated_text'].strip()
        # print(f"Model Output:\n{output}\n")
        # return output
        try:
            lines = [line.strip() for line in output.split("\n") if line.strip()]
            if not lines:
                raise ValueError("No valid sentiment line found")
            sentiment_line = lines[0]
            if sentiment_line.startswith("sentiment:"):
                sentiment = sentiment_line.split(":", 1)[1].strip().lower()
            else:
                match = re.search(r"sentiment:\s*([a-zA-Z]+)", lines[-1], re.IGNORECASE)
                if not match:
                    raise ValueError("Response format is incorrect")
                sentiment = match.group(1)
            sentiment = re.sub(r"[^\w\s]", "", sentiment).strip().split(' ')[0]
            # if not sentiment_line.startswith("sentiment:"):
            #     raise ValueError("Response format is incorrect")
            # sentiment = sentiment_line.split(":", 1)[1].strip().lower()
            # sentiment = re.sub(r"[^\w\s]", "", sentiment).strip()
            valid_sentiments = {"very negative", "negative", "neutral", "positive", "very positive"}
            if sentiment not in valid_sentiments:
                raise ValueError(f"Invalid sentiment: {sentiment}")
            return {"sentiment": sentiment}
        except (IndexError, ValueError) as e:
            print(f"Error processing response: {output} | {e}")
            return {"sentiment": "unknown"}

    def emotion_mistral_2(self, event):
        prompt = f"""
        You are an expert in psychological and affective computing, analyze the following '{event}' and determine the dominant emotion it evokes.
        Reply with strictly one emotion(lowercased, with no quotes, and no punctuations) chosen from this list: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, or neutral.
        Response should be in the following structured format (strictly one line):
        emotion: chosen emotion
        Ensure that the response strictly follows this format. Do not Include any additional explanation, commentary, or justification.
        Do not generate lists, or multiple emotions, or synonyms. No other response will be accepted."""
        response = mistral_pipeline(prompt)
        output = response[0]['generated_text'].strip()
        print(f"\nModel Output:\n{output}\n")
        try:
            lines = [line.strip() for line in output.split("\n") if line.strip()]
            if not lines:
                raise ValueError("No valid sentiment line found")
            emotion_line = lines[0]
            if emotion_line.startswith("emotion:"):
                emotion = emotion_line.split(":", 1)[1].strip().lower()
            else:
                match = re.search(r"sentiment:\s*([a-zA-Z]+(?:\s[a-zA-Z]+)?)", lines[-1], re.IGNORECASE)
                if not match:
                    raise ValueError("Response format is incorrect")
                emotion = match.group(1)
            emotion = re.sub(r"[^\w\s]", "", emotion).strip().split(' ')[0] #remove punctuation

            valid_emotions = {"admiration", "amusement", "anger", "annoyance", "approval", "caring",
                              "confusion", "curiosity", "desire", "disappointment", "disapproval", "disgust",
                              "embarrassment", "excitement", "fear", "gratitude", "grief", "joy", "love",
                              "nervousness", "optimism", "pride", "realization", "relief", "remorse", "sadness",
                              "surprise", "neutral"}
            emotion_mapping = {
            "anxiety": "nervousness",
            "urgency": "fear",
            "acceptance": 'approval',
            "concern": "fear",
            "nostalgia": "realization",
            "unease": "nervousness",
            "dismay": "disappointment",
            "shock": "surprise",
            "resignation": "sadness",
            "empathy": "caring",
            "compassion": "caring",
            "compassionate": "caring",
            "understanding": "caring",
            'appreciation' : "gratitude",
                "respect" : "admiration",
            "determination": "optimism"}
            if emotion not in valid_emotions:
                if emotion in emotion_mapping:
                    mapped_emotion = emotion_mapping[emotion]
                    print(f"for Mapping '{emotion}' to standardized label: '{mapped_emotion}'")
                    emotion = mapped_emotion
                else:
                    raise ValueError(f"Invalid emotion: {emotion}")

            return {"emotion": emotion}

        except (IndexError, ValueError) as e:
            print(f"Error processing response: {output} | {e}")
            return {"emotion": "unknown"}

hate_eval_df = pd.read_csv('hate_eval_dataset')
conan_df = pd.read_csv('conan_dataset')

def main():
    llm_prompt = LLM_sentiment()
    dataset_mistral = pd.read_csv('/content/drive/MyDrive/conan/conan_all_prompt_gpt_sent_emot.csv')
    tqdm.pandas()
    #example
    dataset_mistral['gpt_ngoemotion_sentiment_mistral'] = dataset['hateeval_gpt40mini_ngoemotion_prompt'].progress_apply(lambda x: pd.Series(llm_prompt.sentiment_mistral_2(x)))
    dataset_mistral['gpt_ngoemotion_emotion_mistral'] = dataset['hateeval_gpt40mini_ngoemotion_prompt'].progress_apply(lambda x: pd.Series(llm_prompt.emotion_mistral_2(x)))

    file_name = f'hateeval_gpt_ngoemotion_mistral_sentemot_df.csv'
    file_path = os.path.join(output_path , file_name)
    dataset_mistral.to_csv(file_path, index=False)
    print(f"Saved hateeval_gpt_ngoemotion dataset to : {file_path}\n")
    return dataset_mistral
if __name__ == "__main__":
  dataset_mistral = main()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Reprocessing Unknown Emotions:   0%|          | 0/1 [33:35<?, ?it/s]
Device set to use cuda:0


'def main():\n    llm_prompt = LLM_sentiment()\n\n    # dataset = hate_eval_ngoemotion[[\'text\', \'hateeval_gpt40mini_ngoemotion_prompt\']]\n    # dataset_mistral = hate_eval_vanilla[[\'text\']]\n    dataset_mistral = pd.read_csv(\'/content/drive/MyDrive/PhD with Rawat/WOAH_2025/conan/conan_all_prompt_gpt_sent_emot.csv\')\n    # invalid_rows = dataset_mistral[(dataset_mistral[emotion_column] == \'unknown\')]\n\n    # for item in col_names:\n    tqdm.pandas()\n    dataset_mistral[\'gpt_ngoemotion_sentiment_mistral\'] = dataset[\'hateeval_gpt40mini_ngoemotion_prompt\'].progress_apply(lambda x: pd.Series(llm_prompt.sentiment_mistral_2(x)))\n    dataset_mistral[\'gpt_ngoemotion_emotion_mistral\'] = dataset[\'hateeval_gpt40mini_ngoemotion_prompt\'].progress_apply(lambda x: pd.Series(llm_prompt.emotion_mistral_2(x)))\n\n    file_name = f\'hateeval_gpt_ngoemotion_mistral_sentemot_df.csv\'\n    file_path = os.path.join(output_path , file_name)\n    dataset_mistral.to_csv(file_path, index=Fals